<a href="https://colab.research.google.com/github/dohyeonkim2526/BigContest/blob/master/%EB%B9%85%EC%BD%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
import os
import numpy as np
import pandas as pd

base_dir='/content/gdrive/My Drive/빅데이터 공모전/빅콘테스트 데이터/제공데이터'
sales_data='19년도-실적데이터.xlsx'
sales_dir=os.path.join(base_dir, sales_data)

base_dir_2='/content/gdrive/My Drive/빅데이터 공모전/빅콘테스트 데이터/평가데이터'
pred_data='20년6월-실적예측데이터.xlsx'
pred_dir=os.path.join(base_dir_2, pred_data)

# [예측 데이터]

In [3]:
df_preds=pd.read_excel(pred_dir,header=1)
df_preds  # 예측 상품 데이터프레임

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액
0,2020-06-01 06:20:00,20.000000,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,NaN
1,2020-06-01 06:40:00,20.000000,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,NaN
2,2020-06-01 07:00:00,20.000000,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,NaN
3,2020-06-01 07:20:00,20.000000,100445,202278,쿠미투니카 쿨 레이시 란쥬쉐이퍼&팬티,속옷,69900,NaN
4,2020-06-01 07:40:00,20.000000,100445,202278,쿠미투니카 쿨 레이시 란쥬쉐이퍼&팬티,속옷,69900,NaN
...,...,...,...,...,...,...,...,...
2886,2020-07-01 00:20:00,20.000000,100660,201989,쉴렉스 안마의자 렌탈서비스,무형,0,NaN
2887,2020-07-01 00:40:00,20.000000,100660,201989,쉴렉스 안마의자 렌탈서비스,무형,0,NaN
2888,2020-07-01 01:00:00,20.000000,100660,201989,쉴렉스 안마의자 렌탈서비스,무형,0,NaN
2889,2020-07-01 01:20:00,20.000000,100261,200875,아놀드파마 티셔츠레깅스세트,의류,69900,NaN


In [4]:
df_preds=df_preds.drop(df_preds[(df_preds['판매단가']==0)].index) # 예측 상품 판매가 0원인 것은 예측에서 제외
df_preds=df_preds.reset_index(drop=True)

In [5]:
# 예측 상품의 마더코드 리스트 생성

pred_code=[]
for code in df_preds['마더코드']:
  if code not in pred_code:
    pred_code.append(code)

pred_code.sort()

# [ 2019 판매 데이터 ]

In [6]:
df_sales=pd.read_excel(sales_dir,header=1, na_values='NaN')

In [ ]:
df_sales.dtypes

방송일시     object
노출(분)    object
마더코드     object
상품코드     object
상품명      object
상품군      object
판매단가     object
취급액      object
dtype: object

In [7]:
df_sales.방송일시=pd.to_datetime(df_sales.방송일시, format='%Y%m%d %H:%M:%S')
df_sales['month']=df_sales.방송일시.dt.strftime('%m')
df_sales['week day']=df_sales['방송일시'].dt.day_name()
df_sales['broad date']=df_sales['방송일시'].dt.date
df_sales['broad time']=df_sales['방송일시'].dt.time
df_sales=df_sales.drop('방송일시', axis=1)

In [8]:
df_sales=df_sales.drop(df_sales[(df_sales['week day'] == 'Saturday')].index) # 매주 토요일을 추정에서 제외

In [ ]:
df_sales['broad time']

1        06:00:00
2        06:00:00
3        06:20:00
4        06:20:00
5        06:40:00
           ...   
38300    23:40:00
38301    00:00:00
38302    00:00:00
38303    00:00:00
38304    00:00:00
Name: broad time, Length: 32190, dtype: object

In [9]:
df_sales=df_sales.reset_index(drop=True)

In [ ]:
df_sales

,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,month,week day,broad date,broad time
0,20.000000,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000.0,01,Tuesday,2019-01-01,06:00:00
1,NaN,100346,201079,테이트 여성 셀린니트3종,의류,39900,4371000.0,01,Tuesday,2019-01-01,06:00:00
2,20.000000,100346,201072,테이트 남성 셀린니트3종,의류,39900,3262000.0,01,Tuesday,2019-01-01,06:20:00
3,NaN,100346,201079,테이트 여성 셀린니트3종,의류,39900,6955000.0,01,Tuesday,2019-01-01,06:20:00
4,20.000000,100346,201072,테이트 남성 셀린니트3종,의류,39900,6672000.0,01,Tuesday,2019-01-01,06:40:00
...,...,...,...,...,...,...,...,...,...,...,...
32836,20.000000,100073,200196,삼성화재 행복한파트너 주택화재보험(1912),무형,0,NaN,01,Wednesday,2020-01-01,00:20:00
32837,20.000000,100073,200196,삼성화재 행복한파트너 주택화재보험(1912),무형,0,NaN,01,Wednesday,2020-01-01,00:40:00
32838,20.000000,100073,200196,삼성화재 행복한파트너 주택화재보험(1912),무형,0,NaN,01,Wednesday,2020-01-01,01:00:00
32839,20.000000,100490,201478,더케이 예다함 상조서비스(티포트),무형,0,NaN,01,Wednesday,2020-01-01,01:20:00


In [ ]:
#토요일 18:00-18:20 추정에서 제외(토요일은 제외시켰음)
#심야시간대 추정에서 제외

In [10]:
# 2019 판매 상품의 마더코드 리스트 생성

sales_code=[]
for code in df_sales['마더코드']:
  if code not in sales_code:
    sales_code.append(code)

sales_code.sort()

## [2019 판매/예측 마더코드의 비교→예측 데이터에서 과거 실적이 없는 상품들을 찾아낸다.]

In [11]:
no_sales_code=[]  # 예측 데이터 기준: 과거 실적이 없는 마더코드 리스트
for pred in pred_code:
  if pred not in sales_code:
    no_sales_code.append(pred)

In [12]:
no_sales_code.sort()

In [13]:
df_only_in_preds=pd.DataFrame()

for code in no_sales_code:
  code_preds=df_preds[(df_preds['마더코드']==code)]
  df_only_in_preds=pd.concat([df_only_in_preds,code_preds])

df_only_in_preds=df_only_in_preds.reset_index(drop=True)

In [ ]:
df_only_in_preds  # 과거 실적이 없는 예측 상품 데이터프레임



## [ 과거 실적이 없는 경우 분석 ]



* 예측 데이터 기준 과거 실적이 없는 경우 → 19년도 판매 데이터에 해당 제품의 마더 코드가 없음을 의미한다.

In [15]:
df_only_in_preds

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액
0,2020-06-25 08:20:00,20.0,100003,200004,PN 메가티타늄 후라이팬 기본세트,주방,68000,NaN
1,2020-06-25 08:20:00,NaN,100003,200003,PN 메가티타늄 후라이팬+IH 인덕션 1구 풀세트,주방,98000,NaN
2,2020-06-25 08:40:00,20.0,100003,200004,PN 메가티타늄 후라이팬 기본세트,주방,68000,NaN
3,2020-06-25 08:40:00,NaN,100003,200003,PN 메가티타늄 후라이팬+IH 인덕션 1구 풀세트,주방,98000,NaN
4,2020-06-25 09:00:00,20.0,100003,200004,PN 메가티타늄 후라이팬 기본세트,주방,68000,NaN
...,...,...,...,...,...,...,...,...
1192,2020-06-11 11:20:00,NaN,100847,202507,추억의쥐치포55장,농수축,32900,NaN
1193,2020-06-11 11:40:00,20.0,100847,202506,추억의쥐치포110장,농수축,32900,NaN
1194,2020-06-11 11:40:00,NaN,100847,202507,추억의쥐치포55장,농수축,32900,NaN
1195,2020-06-11 12:00:00,20.0,100847,202506,추억의쥐치포110장,농수축,32900,NaN


In [97]:
product_group=df_preds['상품군'].unique()
product_group

array(['의류', '속옷', '이미용', '주방', '농수축', '가전', '생활용품', '가구', '잡화', '건강기능',
       '침구'], dtype=object)

In [106]:
# for 문을 이용해서 아래의 코드들을 하나의 함수로 구현하기 ... 코드 구현에 대한 방법이 떠오르지 않음
의류_list=[]

for i in range(len(df_preds)):
  pro=df_preds.loc[i]['상품군']
  pro_name=df_preds.loc[i]['상품명']
  if pro=='의류':
    의류_list.append(pro_name)

In [108]:
속옷_list=[]

for i in range(len(df_preds)):
  pro=df_preds.loc[i]['상품군']
  pro_name=df_preds.loc[i]['상품명']
  if pro=='속옷':
    속옷_list.append(pro_name)

In [109]:
이미용_list=[]

for i in range(len(df_preds)):
  pro=df_preds.loc[i]['상품군']
  pro_name=df_preds.loc[i]['상품명']
  if pro=='이미용':
    이미용_list.append(pro_name)

In [111]:
농수축_list=[]

for i in range(len(df_preds)):
  pro=df_preds.loc[i]['상품군']
  pro_name=df_preds.loc[i]['상품명']
  if pro=='농수축':
    농수축_list.append(pro_name)

In [112]:
가전_list=[]

for i in range(len(df_preds)):
  pro=df_preds.loc[i]['상품군']
  pro_name=df_preds.loc[i]['상품명']
  if pro=='가전':
    가전_list.append(pro_name)

In [113]:
생활용품_list=[]

for i in range(len(df_preds)):
  pro=df_preds.loc[i]['상품군']
  pro_name=df_preds.loc[i]['상품명']
  if pro=='생활용품':
    생활용품_list.append(pro_name)

In [114]:
가구_list=[]

for i in range(len(df_preds)):
  pro=df_preds.loc[i]['상품군']
  pro_name=df_preds.loc[i]['상품명']
  if pro=='가구':
    가구_list.append(pro_name)

In [115]:
잡화_list=[]

for i in range(len(df_preds)):
  pro=df_preds.loc[i]['상품군']
  pro_name=df_preds.loc[i]['상품명']
  if pro=='잡화':
    잡화_list.append(pro_name)

In [116]:
가구_list=[]

for i in range(len(df_preds)):
  pro=df_preds.loc[i]['상품군']
  pro_name=df_preds.loc[i]['상품명']
  if pro=='가구':
    가구_list.append(pro_name)

In [117]:
건강기능_list=[]

for i in range(len(df_preds)):
  pro=df_preds.loc[i]['상품군']
  pro_name=df_preds.loc[i]['상품명']
  if pro=='건강기능':
    건강기능_list.append(pro_name)

In [118]:
침구_list=[]

for i in range(len(df_preds)):
  pro=df_preds.loc[i]['상품군']
  pro_name=df_preds.loc[i]['상품명']
  if pro=='침구':
    침구_list.append(pro_name)

In [122]:
# 이 부분도 하나의 함수로 구현하기
pro_set=set(의류_list)
pro_list1=list(pro_set)

pro_set=set(속옷_list)
pro_list2=list(pro_set)

pro_set=set(이미용_list)
pro_list3=list(pro_set)

pro_set=set(주방_list)
pro_list4=list(pro_set)

pro_set=set(농수축_list)
pro_list5=list(pro_set)

pro_set=set(가전_list)
pro_list6=list(pro_set)

pro_set=set(생활용품_list)
pro_list7=list(pro_set)

pro_set=set(가구_list)
pro_list8=list(pro_set)

pro_set=set(잡화_list)
pro_list9=list(pro_set)

pro_set=set(건강기능_list)
pro_list10=list(pro_set)

pro_set=set(침구_list)
pro_list11=list(pro_set)

In [123]:
# 과거 실적이 없는 경우: 상품군에 따른 각 상품명들 확인해보기

['크리스티나앤코 시스루앙상블',
 '라라쎄 자수 레이스 롱원피스 패키지(원피스 2종+볼레로 1종)',
 '로질리 도로시 원피스 3종',
 'CERINI by PAT 여성 올데이 기능성 반팔 티셔츠8종',
 '마르엘라로사티 린넨 베스트 세트[3월]',
 '마르조 로아벨 시어서커원피스 3종',
 '마모트 액티브 하트 라인핏 5종',
 '아놀드파마 티셔츠레깅스세트',
 '르까프 남성 라운드 티셔츠 4종 + 반바지 1종',
 '코몽트 남성 티셔츠 8종(시즌1)',
 '마르엘라로사티 플로럴썸머원피스3종',
 'CERINI by PAT 남성 에어홀 팬츠3종',
 '라라쎄 쉬폰 롤업재킷 1종',
 '코몽트 이지웨어 상하의6종세트',
 '할리마리 풍기인견 원피스 2종',
 '자연감성 남성 셔츠2종',
 'NNF 텐셀팬츠3종',
 '헨리코튼골프 여성 반바지3종세트',
 '잭필드 남성  반팔셔츠 4종',
 '예쎄 여성 썸머 풀코디 4종세트',
 '자연감성 여성블라우스2종',
 '마르엘라로사티SS와이드팬츠',
 'CERINI by PAT 남성 올데이 기능성 반팔 티셔츠8종',
 '코몽트 남성 반팔 티셔츠 8종(시즌2)',
 '르까프 여성 라운드 티셔츠 4종 + 반바지 1종',
 '젠트웰 남성 썸머티셔츠 4종',
 '예작 남성 썸머셔츠 3종',
 '헨리코튼골프 남성 반바지3종세트',
 '르까프 여성 썸머 쿨카라 티셔츠 5종']